In [19]:
import os 
from dotenv import load_dotenv
from pathlib import Path

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, CodeInterpreterTool, CodeInterpreterToolAuto

In [20]:
#import endpoint and deployed model name
load_dotenv()

endpoint = os.environ['PROJECT_ENDPOINT']
model = os.environ['MODEL_DEPLOYMENT_NAME']

In [21]:
#create the project client
credential = DefaultAzureCredential()
project_client = AIProjectClient(
    endpoint=endpoint,
    credential=credential
)
project_client

In [22]:
#create the chat client
chat_client = project_client.get_openai_client()
chat_client

In [23]:
#upload the data file and create code interpreter tool
current_dir = Path.cwd()
file_path = current_dir / 'data.txt'

file = chat_client.files.create(
    file=open(file_path, 'rb'),
    purpose='assistants'
)
print(f"Uploaded {file.filename}")

#code interpreter tool
code_interpreter = CodeInterpreterTool(
    container=CodeInterpreterToolAuto(file_ids=[file.id])
)

Uploaded data.txt


In [24]:
#define an agent that uses code interpretertool
agent = project_client.agents.create_version(
    agent_name='data-agent',
    definition=PromptAgentDefinition(
        model=model,
        instructions='You are an AI agent that analyzes the data in the file that has been uploaded. Use Python to calculate statistical metrics as necessary.',
        tools=[code_interpreter],
    ),
)
print(f"Using agent: {agent.name}")

Using agent: data-agent


In [25]:
#create a thread for the conversation
conversation = chat_client.conversations.create()

In [27]:
#loop the user conversations until the user types quit

while True:
    user_prompt = input(f"Enter a prompt to get response (or type 'quit' to exit the conversation)")

    if user_prompt.lower() == 'quit':
        break

    if len(user_prompt) == 0:
        continue

    #send the prompt to the agent
    chat_client.conversations.items.create(
        conversation_id=conversation.id,
        items=[
            {
                'type':'message',
                'role':'user',
                'content':user_prompt,
            }
        ]
    )

    #get the response from the agent
    response = chat_client.responses.create(
        conversation=conversation.id,
        extra_body={
            'agent':{
                'name':agent.name,
                'type':'agent_reference'
            }
        },
        input="",
    )

    #check the response status for failure
    if response.status == 'failed':
        print(f"Response failed: {response.error}")

    #fetch the latest conversation items
    items = chat_client.conversations.items.list(conversation_id=conversation.id)

    #show the latest response from the agent
    last_msg = None
    for item in items.data:
        if item.type == 'message' and item.role == 'assistant':
            last_msg = item
            break

    if last_msg:
        print("\nAssistant:", last_msg.content[0].text)
    else:
        print('No assistants messages are found')


Assistant: The standard deviation of the costs is **1022.47**.  

The category with the highest cost is **Transportation**, with a cost of **2301.0**.

Assistant: Here is the updated text-based bar chart showing the cost by category:

```
Accommodation  : ██████████████ 674.56
Transportation : ██████████████████████████████████████████████████ 2301.00
Meals          : █████ 267.89
Misc.          :  34.50
```

Let me know if you need further assistance or modifications.

Assistant: The standard deviation of the costs is **1022.47**.


In [28]:
#finally print the conversation history
print("\nCoversation Log:\n")

all_items = chat_client.conversations.items.list(conversation_id=conversation.id)

for each_item in reversed(all_items.data):
    if each_item.type == 'message':
        role = each_item.role.upper()
        content = each_item.content[0].text
        print(f"{role}: {content}\n")


Coversation Log:

ASSISTANT: The category with the highest cost is **Transportation**, with a cost of **2301.0**.

USER: Create a text-based bar chart showing cost by category

ASSISTANT: You seem to have reuploaded the file. Let me reanalyze its structure and create the text-based bar chart for the cost by category.

ASSISTANT: Now that the data structure is understood, I'll create a text-based bar chart showing the cost for each category.

ASSISTANT: Here is the text-based bar chart showing the cost by category:

```
Accommodation  : ██████████████ 674.56
Transportation : ██████████████████████████████████████████████████ 2301.00
Meals          : █████ 267.89
Misc.          :  34.50
```

The length of the bars corresponds to the relative cost for each category. Let me know if you need further assistance!

USER: What's the standard deviation of cost?

USER: What's the category with the highest cost?

ASSISTANT: The standard deviation of the costs is **1022.47**.  

The category with 

In [29]:
# clean up the conversations and delete the agent
from openai import project


chat_client.conversations.delete(conversation_id=conversation.id)
print("All conversations from the above chat were deleted")

project_client.agents.delete(
    agent_name=agent.name,
)
print("Agent was deleted from the project")

All conversations from the above chat were deleted
Agent was deleted from the project
